### Copy paste below into browser developer console to prevent Google colab timeout error:

function ClickConnect(){ console.log("Connnect Clicked - Start"); document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click(); console.log("Connnect Clicked - End"); }; setInterval(ClickConnect, 60000)

## #Burger data decompress, organize, clean
## Output: df_burger.csv file

In [0]:
import lzma
import json
import pandas as pd
import numpy as np
import re
import glob
import os
import sys

In [0]:
# speed up code
import multiprocessing as mp

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/CKIDS_main/data/food_tags/#burger")

In [9]:
# lists of file names

# descrip txts
txt_files = glob.glob("*_UTC.txt")
# location txts
loc_files = glob.glob("*_location.txt")
# json xz
jsonxz_files = glob.glob("*.json.xz")
# comments jsons
cmt_files = glob.glob("*_comments.json")

print(len(txt_files))
print(len(loc_files))
print(len(jsonxz_files))
print(len(cmt_files))

8837
1310
5388
1927


In [0]:
# a list of dictionaries
jsons = []

for file in jsonxz_files:
  read = lzma.open(file).read().decode('utf-8')
  jsondict = json.loads(read)
  jsons.append(jsondict["node"])  

In [0]:
print(len(jsons))

5388


In [0]:
df = pd.DataFrame(jsons)

In [0]:
# save progress
os.chdir("/content/drive/My Drive/Colab Notebooks/CKIDS_main/data/food_tags")
df.to_csv('burger_df1.csv')

In [0]:
# Using apply and lambda to create new column
# of usernames per post

def get_name(owner):
  return owner['username']

df['username'] = df.apply(lambda x: get_name(x['owner']), axis=1)

In [0]:
# continue last left off
os.chdir("/content/drive/My Drive/Colab Notebooks/CKIDS_main/data/food_tags")
df = pd.read_csv('burger_df1.csv')

## Post Descript Text files
Open all post description txt files to a long list of strings, extract all hashtags from that list, merge the text, hashtags, and filenames lists to df

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/CKIDS_main/data/food_tags/#burger")

In [0]:
def read_txt(file):
  text = open(file, "r").read()
  return text   # return as string

In [0]:
# try multiprocessing for faster code
p = mp.Pool(5)
%time texts = list(p.map(read_txt, txt_files))

CPU times: user 74.1 ms, sys: 48.2 ms, total: 122 ms
Wall time: 23.2 s


extract hashtags as list of lists (of each post's hashtags)

In [0]:
def get_ht(text):
  ht = re.findall(r"#(\w+)", text)
  return ht

In [0]:
p2 = mp.Pool(5)
%time hashtags = list(p2.map(get_ht, texts))
# hashtags = [re.findall(r"#(\w+)", t) for t in texts]

CPU times: user 57.5 ms, sys: 46.5 ms, total: 104 ms
Wall time: 172 ms


In [0]:
# get file names 
txt_fnames = [f.split('.txt')[0] for f in txt_files]
jsonxz_fnames = [f.split('.json.xz')[0] for f in jsonxz_files]

In [0]:
# convert all to series, and concat them to a df
textsS = pd.Series(texts)
hashtagsS = pd.Series(hashtags)
txt_fnamesS = pd.Series(txt_fnames)
df_txt = pd.DataFrame(pd.concat([txt_fnamesS,textsS,hashtagsS],axis=1))
df_txt.columns = ['file_name', 'descrip', 'hashtags']

In [0]:
# concat with the main df
merged = pd.concat([df_txt, df], axis=1, join='outer')
print(merged.shape)

(8837, 51)


In [0]:
merged.head()

,file_name,descrip,hashtags,Unnamed: 0,comments_disabled,__typename,id,edge_media_to_caption,shortcode,edge_media_to_comment,taken_at_timestamp,dimensions,display_url,edge_liked_by,edge_media_preview_like,owner,thumbnail_src,thumbnail_resources,is_video,accessibility_caption,gating_info,fact_check_overall_rating,fact_check_information,media_preview,display_resources,tracking_token,edge_media_to_tagged_user,caption_is_edited,has_ranked_comments,edge_media_to_parent_comment,edge_media_to_hoisted_comment,edge_media_preview_comment,commenting_disabled_for_viewer,edge_media_to_sponsor_user,location,viewer_has_liked,viewer_has_saved,viewer_has_saved_to_collection,viewer_in_photo_of_you,viewer_can_reshare,is_ad,edge_web_media_to_related_media,edge_sidecar_to_children,product_type,video_view_count,dash_info,video_url,encoding_status,is_published,title,video_duration
0,2020-03-06_14-51-33_UTC,Damak tadına giden her yol AnkaPizza\n\nEşi Be...,"[yarışma, 30dkchallenge, ankapizza, ankatavuk,...",0.0,False,GraphImage,2.258952e+18,{'edges': [{'node': {'text': '殘存的義大利甜蜜回憶，在這波武漢...,B9ZaS2aFLj2,{'count': 0},1.583508e+09,"{'height': 1080, 'width': 1080}",https://scontent-sea1-1.cdninstagram.com/v/t51...,{'count': 6},"{'count': 6, 'edges': []}","{'id': '2036437413', 'is_verified': False, 'pr...",https://scontent-sea1-1.cdninstagram.com/v/t51...,[{'src': 'https://scontent-sea1-1.cdninstagram...,False,"Photo by omega_tw🇹🇼✈️🌎 on March 06, 2020.",NaN,NaN,NaN,ACoqv+UKhj+YMwK7UO0D39T7egHWp538pC3oOPr2rDhdon...,[{'src': 'https://scontent-sea1-1.cdninstagram...,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,{'edges': []},True,False,"{'count': 0, 'page_info': {'has_next_page': Fa...",{'edges': []},"{'count': 0, 'edges': []}",False,{'edges': []},NaN,False,False,False,False,True,False,{'edges': []},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-06_14-50-17_UTC,What is the first thing you do after landing i...,"[frysauce, burger, frys, fishfryfriday, 4chion...",1.0,False,GraphImage,2.258952e+18,{'edges': [{'node': {'text': '... toll wenn da...,B9ZaSCwKDgY,{'count': 1},1.583508e+09,"{'height': 1080, 'width': 1080}",https://scontent-sea1-1.cdninstagram.com/v/t51...,{'count': 14},"{'count': 14, 'edges': []}","{'id': '14240247500', 'is_verified': False, 'p...",https://scontent-sea1-1.cdninstagram.com/v/t51...,[{'src': 'https://scontent-sea1-1.cdninstagram...,False,Image may contain: car,NaN,NaN,NaN,ACoqwCCOtKBmrqvHLDskO105Qjvnsfp2qlgY/n/SmBKrEc...,[{'src': 'https://scontent-sea1-1.cdninstagram...,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,{'edges': []},False,False,"{'count': 1, 'page_info': {'has_next_page': Fa...",{'edges': []},"{'count': 1, 'edges': [{'node': {'id': '180199...",False,{'edges': []},NaN,False,False,False,False,True,False,{'edges': []},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-06_14-47-57_UTC,Thursday night rides out to the west side. #bu...,"[burger, thursdaynightriders, BMCseat, myBritt...",2.0,False,GraphImage,2.258952e+18,{'edges': [{'node': {'text': 'Olha que ótima i...,B9ZaOrMgJ9O,{'count': 0},1.583508e+09,"{'height': 1080, 'width': 1080}",https://scontent-sea1-1.cdninstagram.com/v/t51...,{'count': 8},"{'count': 8, 'edges': []}","{'id': '7761513592', 'is_verified': False, 'pr...",https://scontent-sea1-1.cdninstagram.com/v/t51...,[{'src': 'https://scontent-sea1-1.cdninstagram...,False,Image may contain: one or more people and text,NaN,NaN,NaN,ACoqvSkhRwAMDng5yOPTGT/kUsabIjuwMd8An3xV9TiNT2...,[{'src': 'https://scontent-sea1-1.cdninstagram...,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,{'edges': []},False,False,"{'count': 0, 'page_info': {'has_next_page': Fa...",{'edges': []},"{'count': 0, 'edges': []}",False,{'edges': []},NaN,False,False,False,False,True,False,{'edges': []},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-06_14-44-17_UTC,"Aye, @porkyslowfell for the second time this w...","[food, burger]",3.0,False,GraphImage,2.258952e+18,{'edges': [{'node': {'text': 'O corte da pican...,B9ZaM13hJP4,{'count': 1},1.583508e+09,"

In [0]:
# save progress
os.chdir("/content/drive/My Drive/Colab Notebooks/CKIDS_main/data/food_tags")
merged.to_csv('burger_merged.csv')

In [0]:
# load progress
os.chdir("/content/drive/My Drive/Colab Notebooks/CKIDS_main/data/food_tags")
merged = pd.read_csv('burger_merged.csv')

# combine geotag data with main df

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/CKIDS_main/data/food_tags/#burger")

In [11]:
p3 = mp.Pool(5)
%time locs = list(p3.map(read_txt, loc_files))
# locs = [open(file).read() for file in loc_files]

CPU times: user 197 ms, sys: 82.6 ms, total: 279 ms
Wall time: 1min 36s


In [0]:
# get location name 
#by slicing up to first '\n'
loc_names = [s[0:s.find('\n')] for s in locs]

In [0]:
loc_fnames = [f.split('_location.txt')[0] for f in loc_files]
locsS = pd.Series(locs)
loc_namesS = pd.Series(loc_names)
loc_fnamesS = pd.Series(loc_fnames)

In [0]:
df_loc = pd.DataFrame(pd.concat([loc_namesS, locsS, loc_fnamesS],axis=1))
df_loc.columns = ['location_name', 'location_desc', 'file_name']

In [24]:
df_loc.head()

,location_name,location_desc,file_name
0,Small Cheval,Small Cheval\nhttps://maps.google.com/maps?q=4...,2020-03-06_17-15-36_UTC
1,BONEHEAD,BONEHEAD\nhttps://maps.google.com/maps?q=52.47...,2020-03-06_17-14-46_UTC
2,Popular Fresh Momos,Popular Fresh Momos\nhttps://maps.google.com/m...,2020-03-06_17-14-33_UTC
3,Vegas Gastrobar,Vegas Gastrobar\nhttps://maps.google.com/maps?...,2020-03-06_16-53-55_UTC
4,"Bucaramanga, Santander","Bucaramanga, Santander\nhttps://maps.google.co...",2020-03-06_17-38-38_UTC


In [0]:
new_merged = pd.DataFrame.merge(merged, df_loc, how = 'outer', on = 'file_name')

In [0]:
# verify how many posts have loc data
withlocs = new_merged.dropna(subset=["location_name"])
# withlocs.head()

In [34]:
new_merged.shape   # final df shape

(8837, 54)

## Export final df to csv

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/CKIDS_main/data/food_tags")
new_merged.to_csv('df_burger.csv')

# deal with comment files
# put in separate df?

In [36]:
# a list of dictionaries
os.chdir("/content/drive/My Drive/Colab Notebooks/CKIDS_main/data/food_tags/#burger")
cmts = []

for file in cmt_files:
  with open(file) as f:
    jsondict = json.load(f)
    cmts.append(jsondict)

KeyboardInterrupt: ignored

In [0]:
cmt_fnames = [f.split('_comments.json')[0] for f in cmt_files]
cmt_fnamesS = pd.Series(cmt_fnames)

In [0]:
cmtsS = pd.Series(cmts)

In [0]:
df_cmts = pd.DataFrame(pd.concat([cmt_fnamesS, cmtsS],axis=1))
df_cmts.columns = ['file_name', 'comments']

In [0]:
df_cmts.head()

In [0]:
# # example of a comment's data
# df_cmts.iloc[0]['comments']

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/CKIDS_main/data/food_tags")
df_cmts.to_csv('comments_burger.csv')